## Get Eigenvalues Script

Having assembled the stiffness matrices $A_1$ and $A_2$ representing the integrals $\int \nabla u\cdot\nabla\phi \ \mathrm{d}x$ and $\int u\phi \ \mathrm{d}x$ respectively, we now need to compute the eigenvalues $\omega^2$ and eigenvectors $u$ such that
$$ \left( A_1 - \omega^2 A_2 \right)u = 0. $$
**NOTE:** As per `AssembleStiffnessMatrices.py`, the prefactor $-\omega^2$ is not included in $A_2$, so that we can solve the above as a generalised eigenvalue problem using SciPy.

When the equivalent script is run via the command line, the argument to be passed from the command line should be the proportion of eigenvalues to compute, equivalent to the variable `evalFrac` which is manually defined in this notebook.

### Requirements:
- The fenicsproject environment be active
- The stiffness matrices are stored in the subdirectory `./StiffnessMatrixDump/`, under the naming convention `a?_N*.npz` where `?` is either `1` or `2`, and `*` is the value of $N$ as found in the `.geo` file which generates the mesh domain. 
- A folder `./EvalDump/` exists to store the computed eigenvalues and eigenvectors in

### Outputs:
Outputs are placed in the `./EvalDump/` subdirectory.
- File `evals-N*-?` where `*` is replaced by the value of $N$ for the current domain (inferred from files), and `?` is replaced by the number of eigenvalues stored. This is a list of all the eigenvalues $\omega>0$ that were found.
- File `evecs-N*-?` where `*` is replaced by the value of $N$ for the current domain, and `?` is replaced by the number of eigenvectors stored. This is a list of all the corresponding eigenvectors $u$ that were found, each vector being stored as one column.

In [1]:
from fenics import *
from dolfin import *
import matplotlib.pyplot as plt
import scipy.sparse as spSparse
import scipy.sparse.linalg as spla
import numpy as np

In [2]:
# This is the name of our mesh files, minus the extension.
filePrefix = 'FEM_CrossGraphSetup'
# This is the gmsh file which generated our domain's mesh. We can infer information about our domain from it
gmshFile = filePrefix + '.geo'
# This is the folder into which we will place our stiffness matrices, once they are assembled
matDumpFolder = 'StiffnessMatrixDump'
evalDumpFolder = 'EvalDump'

# Deduce value of N from gmshFile - it appears on line 12,
# with the 5th character being (the start of) the value of N.
with open('FEM_CrossGraphSetup.geo') as fp:
    for i, line in enumerate(fp):
        if i == 11:
            # 11th line
            Nstr = line[4:-2] #take all characters after the numerical value appears in the file
        elif i > 11:
            #terminate loop early
            break
# We want N as a float because we will use it in computations, to define points on meshes etc. Int might cause bugs.
for i, char in enumerate(Nstr):
    if char==';':
        Nstr = Nstr[:i]
        break
N = float(Nstr)

# Infer filenames via naming convention
fA1 = './' + matDumpFolder + '/a1_N' + Nstr + '.npz'
fA2 = './' + matDumpFolder + '/a2_N' + Nstr + '.npz'
# Create filename to save evals to
evalFile = './' + evalDumpFolder + './evals-N' + Nstr

In [3]:
# create a subclass for the 2D-periodic domain...
class PeriodicDomain(SubDomain):
    #map left--> right and bottom-->top

    def inside(self, x, on_boundary):
        # return True if on left or bottom boundary AND NOT on one of the two corners
        return bool((near(x[0], 0) or near(x[1], 0)) and
                (not ((near(x[0], 0) and near(x[1], N)) or
                        (near(x[0], N) and near(x[1], 0)))) and on_boundary)

    def map(self, x, y):
        if near(x[0], N) and near(x[1], N):
            y[0] = x[0] - N
            y[1] = x[1] - N
        elif near(x[0], N):
            y[0] = x[0] - N
            y[1] = x[1]
        else:   # near(x[1], N)
            y[0] = x[0]
            y[1] = x[1] - N
#concludes the SubDomain subclass definition

# Now we import our mesh...
meshFile = filePrefix + '.xml'
mesh = Mesh(meshFile)

In [4]:
# Read matrices we want in CSR format
print('Loading stiffness matrices.')
A1 = spSparse.load_npz(fA1)
A2 = spSparse.load_npz(fA2)

# Check that matrices are of the same shape, and are square
if A1.shape != A2.shape:
    raise ValueError('Stiffness matrices have different shapes')
elif A1.shape[0] != A1.shape[1]:
    raise ValueError('Stiffness matrices are non-square')
else:
    nNodes = A1.shape[0] 
# the size of the stiffness matrices corresponds to the number of nodes in our mesh,
# and hence the maximum number of eigenvalues that we can find.

# Could also check sparsity patterns...
#plt.spy(A1)
#plt.show()
#plt.spy(A2)
#plt.show()

Loading stiffness matrices.


In [5]:
# We now need to find the eigenvalues at the bottom of the spectrum.
# To do so, we use Scipy.sparse.linalg's generalised eigenvalue solver,
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html
# We can do this since A1 and A2 should both be symmetric for our problem (we have an elliptic operator).
# They might even be better behaved, but I don't think we need that here.

# First, we can never find *all* the eigenvalues, and it would be super inefficient to try to do that too.
# So let's just try to compute a fraction of them.
evalFrac = 0.1
evalsToCompute = int(np.ceil(nNodes * evalFrac))
# We want to compute at most evalFrac (as a percentage) eigenvalues out of nNodes
print('Computing %d eigenvalues closest to 0' % (evalsToCompute))

# Solves A1 * x = lambda * A2 * x for an eigenpair (lambda, x).
# Eigenvectors are stored column-wise, so x[:,i] is the eigenvector of lambda[i].
# Also, we use lambda = omega^2 here.
lambdaVals, wVecs = spla.eigsh(A1, k = evalsToCompute, M=A2, sigma = 0.0, which='LM', return_eigenvectors = True)

Computing 72 eigenvalues closest to 0


In [6]:
# Check if any "eigenvalues" that were computed were below 0, and deal with them accordingly
nNegEvals = len(lambdaVals[lambdaVals<0])
if nNegEvals>0:
    print('Found %d negative eigenvalues, the largest of which is: %.5e - these are being removed.' \
          % (nNegEvals, np.min(lambdaVals[lambdaVals<0])) )
else:
    print('No negative eigenvalues found.')
    
# For safety, we should really discard these "negative" eigenvalues
wVecs = wVecs[:, lambdaVals>=0]
lambdaVals = lambdaVals[lambdaVals>=0]

# Now we can save the legitimate eigenvalues (taking the square-root too to obtain omegas)
# And can also save their eigenvectors
evalSaveStr = './' + evalDumpFolder + '/evals-N' + Nstr + '-' + str(int(evalsToCompute - nNegEvals))
evecSaveStr = './' + evalDumpFolder + '/evecs-N' + Nstr + '-' + str(int(evalsToCompute - nNegEvals))

wVals = np.sqrt(lambdaVals) #since we've removed the <0 values, sqrt is safe
# Save (omega, u) pairs to output folder
np.save(evalSaveStr, wVals)
np.save(evecSaveStr, wVecs)
print('Saved to output files %s (evals) and %s (evecs)' % (evalSaveStr, evecSaveStr))

No negative eigenvalues found.
Saved to output files ./EvalDump/evals-N3-72 (evals) and ./EvalDump/evecs-N3-72 (evecs)


In [9]:
# This is here for reference when creating plots, it won't be executed by "running all" by default.
if 0==1:
    # To plot functions, we need to know the function space we are working with, which we obtain from the mesh
    # Although the data for the eigenvectors is stored in wVecs, in order to correctly plot it and assign values at
    # each node in the mesh, we need the topographical data from the mesh and the function space we are using
    V = FunctionSpace(mesh, 'CG', 1, constrained_domain=PeriodicDomain())

    # We can now plot each of the eigenfunctions that we found, and display it's corresponding eigenvalue
    for i, w in enumerate(wVals):
        # Assemble solution from eigenvector that was found
        u = Function(V)
        u.vector()[:] = wVecs[:,i] #insert data from wVecs into function-space object
        # Plot solution...
        p = plot(u)
        p.set_cmap("viridis")
        plt.colorbar(p)
        plt.xlabel('$x$')
        plt.ylabel('$y$')
        plt.title('$ \omega= %.5e $' % (wVals[i]))
        plt.show()

#plt.savefig("PrettyPicture.png") #saves a blank screen for some reason - yeah, because .show() clears everything!

Proceed with plots? (0/1):0
